## 4Geeks Water Analytics Project

In [20]:
# Import dependencies
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error

In [7]:
# Import datasets
aquifer_auser_df = pd.read_csv("./data/Aquifer_Auser.csv")
aquifer_petrignano_df = pd.read_csv("./data/Aquifer_Petrignano.csv")
aquifer_doganella_df = pd.read_csv("./data/Aquifer_Doganella.csv")
aquifer_luco_df = pd.read_csv("./data/Aquifer_Luco.csv")

In [8]:
# Set the option to display all rows
pd.set_option('display.max_columns', None)

In [9]:
# Data Cleaning

# Drop any rows with NaN
aquifer_auser_df = aquifer_auser_df.dropna()
aquifer_petrignano_df = aquifer_petrignano_df.dropna()
aquifer_doganella_df = aquifer_doganella_df.dropna()
aquifer_luco_df = aquifer_luco_df.dropna()

# Reset Indexes
aquifer_auser_df = aquifer_auser_df.reset_index(drop=True)
aquifer_petrignano_df = aquifer_petrignano_df.reset_index(drop=True)
aquifer_doganella_df = aquifer_doganella_df.reset_index(drop=True)
aquifer_luco_df = aquifer_luco_df.reset_index(drop=True)

# Set the 'Date' column as the index of the DataFrames
aquifer_auser_df = aquifer_auser_df.set_index('Date')
aquifer_petrignano_df= aquifer_petrignano_df.set_index('Date')
aquifer_doganella_df = aquifer_doganella_df.set_index('Date')
aquifer_luco_df = aquifer_luco_df.set_index('Date')

In [15]:
# Split the datasets into train and test sets

def train_test_split(data, n=10):
    """Splits the data into train and test sets, with the last n rows as the test set."""
    train = data.iloc[:-n]  # All rows except the last n rows
    test = data.iloc[-n:]   # The last n rows
    return train, test


# Splitting each dataset
aquifer_auser_train_df, aquifer_auser_test_df = train_test_split(aquifer_auser_df)
aquifer_petrignano_train_df, aquifer_petrignano_test_df = train_test_split(aquifer_petrignano_df)
aquifer_doganella_train_df, aquifer_doganella_test_df = train_test_split(aquifer_doganella_df)
aquifer_luco_train_df, aquifer_luco_test_df = train_test_split(aquifer_luco_df)

In [18]:
# AQUIFER AUSER

# 1. Train the VAR model on the entire DataFrame
aquifer_auser_model = VAR(aquifer_auser_df)
aquifer_auser_model_fitted = aquifer_auser_model.fit(maxlags=15, ic='aic')  # Example: using AIC to select optimal lag

# 2. Use the last 'lag_order' observations from the entire DataFrame for forecasting
aquifer_auser_lag_order = aquifer_auser_model_fitted.k_ar
aquifer_auser_last_obs = aquifer_auser_df.values[-aquifer_auser_lag_order:]

# Forecast the next 10 steps using the entire dataset
aquifer_auser_forecast = aquifer_auser_model_fitted.forecast(aquifer_auser_last_obs, steps=10)

# 3. Generate the forecast index
aquifer_auser_forecast_index = pd.date_range(start=aquifer_auser_train_df.index[-1], periods=10, freq='D')

# Convert the forecast to a DataFrame for easier interpretation
aquifer_auser_forecast_df = pd.DataFrame(aquifer_auser_forecast, index=aquifer_auser_forecast_index, columns=aquifer_auser_train_df.columns)

# Extract forecasts for the variables of interest
aquifer_auser_predictions = aquifer_auser_forecast_df[['Depth_to_Groundwater_SAL', 'Depth_to_Groundwater_CoS', 'Depth_to_Groundwater_LT2']]
print(aquifer_auser_predictions)

/home/codespace/.python/current/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:559: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  _index = to_datetime(index)
/home/codespace/.python/current/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


            Depth_to_Groundwater_SAL  Depth_to_Groundwater_CoS  \
2020-06-20                 -5.341536                 -6.331577   
2020-06-21                 -5.573894                 -5.689308   
2020-06-22                 -4.033962                 -4.126749   
2020-06-23                 -5.715087                 -6.340886   
2020-06-24                 -7.497582                 -7.874013   
2020-06-25                 -4.863581                 -4.082048   
2020-06-26                 -4.620977                 -4.752809   
2020-06-27                 -5.673401                 -5.724208   
2020-06-28                 -5.070341                 -5.517067   
2020-06-29                 -6.381386                 -5.765203   

            Depth_to_Groundwater_LT2  
2020-06-20                -15.504651  
2020-06-21                -13.534632  
2020-06-22                -10.974251  
2020-06-23                -11.305584  
2020-06-24                -12.986009  
2020-06-25                -13.696100  


In [22]:
# EVALUATE AQUIFER AUSER MODEL

# Columns of interest
columns_of_interest = ['Depth_to_Groundwater_SAL', 'Depth_to_Groundwater_CoS', 'Depth_to_Groundwater_LT2']

# Initialize an empty dictionary to store MSE for each column
mse_scores = {}

# Loop over each column of interest to calculate MSE
for column in columns_of_interest:
    # Extract the true values from the test set and the predicted values
    true_values = aquifer_auser_test_df[column][:10]  # Only the first 5 values to match the forecast length
    predicted_values = aquifer_auser_predictions[column]
    
    # Calculate MSE and store it in the dictionary
    mse = mean_squared_error(true_values, predicted_values)
    mse_scores[column] = mse

# Print MSE scores
for column, mse in mse_scores.items():
    print(f"MSE for {column}: {mse}")

MSE for Depth_to_Groundwater_SAL: 3.085480717217324
MSE for Depth_to_Groundwater_CoS: 2.6232598390371686
MSE for Depth_to_Groundwater_LT2: 1.811274162250081


In [19]:
# AQUIFER PETRIGNANO


# 1. Train the VAR model on the entire DataFrame
aquifer_petrignano_model = VAR(aquifer_petrignano_df)
aquifer_petrignano_model_fitted = aquifer_petrignano_model.fit(maxlags=15, ic='aic')  # Example: using AIC to select optimal lag

# 2. Use the last 'lag_order' observations from the entire DataFrame for forecasting
aquifer_petrignano_lag_order = aquifer_petrignano_model_fitted.k_ar
aquifer_petrignano_last_obs = aquifer_petrignano_df.values[-aquifer_petrignano_lag_order:]

# Forecast the next 10 steps using the entire dataset
aquifer_petrignano_forecast = aquifer_petrignano_model_fitted.forecast(aquifer_petrignano_last_obs, steps=10)

# 3. Generate the forecast index
aquifer_petrignano_forecast_index = pd.date_range(start=aquifer_petrignano_train_df.index[-1], periods=10, freq='D')

# Convert the forecast to a DataFrame for easier interpretation
aquifer_petrignano_forecast_df = pd.DataFrame(aquifer_petrignano_forecast, index=aquifer_petrignano_forecast_index, columns=aquifer_petrignano_train_df.columns)

# Extract forecasts for the variables of interest
aquifer_petrignano_predictions = aquifer_petrignano_forecast_df[['Depth_to_Groundwater_P24', 'Depth_to_Groundwater_P25']]
print(aquifer_petrignano_predictions)

/home/codespace/.python/current/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


            Depth_to_Groundwater_P24  Depth_to_Groundwater_P25
2020-06-20                -25.944056                -25.287684
2020-06-21                -25.944328                -25.303637
2020-06-22                -25.917593                -25.305391
2020-06-23                -25.940359                -25.323843
2020-06-24                -25.951014                -25.345301
2020-06-25                -25.964361                -25.358629
2020-06-26                -25.997857                -25.381939
2020-06-27                -26.015380                -25.403068
2020-06-28                -26.030003                -25.417891
2020-06-29                -26.041073                -25.431660


In [23]:
# EVALUATE AQUIFER PETIGNANO MODEL

# Columns of interest
columns_of_interest = ['Depth_to_Groundwater_P24', 'Depth_to_Groundwater_P25']

# Initialize an empty dictionary to store MSE for each column
mse_scores = {}

# Loop over each column of interest to calculate MSE
for column in columns_of_interest:
    # Extract the true values from the test set and the predicted values
    true_values = aquifer_petrignano_test_df[column][:10]  # Only the first 5 values to match the forecast length
    predicted_values = aquifer_petrignano_predictions[column]
    
    # Calculate MSE and store it in the dictionary
    mse = mean_squared_error(true_values, predicted_values)
    mse_scores[column] = mse

# Print MSE scores
for column, mse in mse_scores.items():
    print(f"MSE for {column}: {mse}")

MSE for Depth_to_Groundwater_P24: 0.13135113285984085
MSE for Depth_to_Groundwater_P25: 0.11172622993178923


In [26]:
# AQUIFER DOGANELLA


# 1. Train the VAR model on the entire DataFrame
aquifer_doganella_model = VAR(aquifer_doganella_df)
aquifer_doganella_model_fitted = aquifer_doganella_model.fit(ic='aic')  # Example: using AIC to select optimal lag

# 2. Use the last 'lag_order' observations from the entire DataFrame for forecasting
aquifer_doganella_lag_order = aquifer_doganella_model_fitted.k_ar
aquifer_doganella_last_obs = aquifer_doganella_df.values[-aquifer_doganella_lag_order:]

# Forecast the next 10 steps using the entire dataset
aquifer_doganella_forecast = aquifer_doganella_model_fitted.forecast(aquifer_doganella_last_obs, steps=10)

# 3. Generate the forecast index
aquifer_doganella_forecast_index = pd.date_range(start=aquifer_doganella_train_df.index[-1], periods=10, freq='D')

# Convert the forecast to a DataFrame for easier interpretation
aquifer_doganella_forecast_df = pd.DataFrame(aquifer_doganella_forecast, index=aquifer_doganella_forecast_index, columns=aquifer_doganella_train_df.columns)

# Extract forecasts for the variables of interest
aquifer_doganella_predictions = aquifer_doganella_forecast_df[['Depth_to_Groundwater_Pozzo_1', 'Depth_to_Groundwater_Pozzo_2', 'Depth_to_Groundwater_Pozzo_3', 'Depth_to_Groundwater_Pozzo_4','Depth_to_Groundwater_Pozzo_5','Depth_to_Groundwater_Pozzo_6', 'Depth_to_Groundwater_Pozzo_7', 'Depth_to_Groundwater_Pozzo_8', 'Depth_to_Groundwater_Pozzo_9']]
print(aquifer_doganella_predictions)

            Depth_to_Groundwater_Pozzo_1  Depth_to_Groundwater_Pozzo_2  \
2020-06-20                    -50.416486                   -100.354440   
2020-06-21                    -50.982017                   -100.223708   
2020-06-22                    -50.949044                    -99.556351   
2020-06-23                    -50.795120                    -99.634133   
2020-06-24                    -51.209145                    -99.774158   
2020-06-25                    -52.033736                    -99.842102   
2020-06-26                    -51.647550                    -99.623178   
2020-06-27                    -51.183951                    -99.564621   
2020-06-28                    -50.700186                    -99.607430   
2020-06-29                    -50.396130                    -99.533367   

            Depth_to_Groundwater_Pozzo_3  Depth_to_Groundwater_Pozzo_4  \
2020-06-20                   -102.474354                   -101.377427   
2020-06-21                   -101.214

/home/codespace/.python/current/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [27]:
# EVALUATE AQUIFER DOGANELLA MODEL

# Columns of interest
columns_of_interest = ['Depth_to_Groundwater_Pozzo_1', 'Depth_to_Groundwater_Pozzo_2', 'Depth_to_Groundwater_Pozzo_3', 'Depth_to_Groundwater_Pozzo_4','Depth_to_Groundwater_Pozzo_5','Depth_to_Groundwater_Pozzo_6', 'Depth_to_Groundwater_Pozzo_7', 'Depth_to_Groundwater_Pozzo_8', 'Depth_to_Groundwater_Pozzo_9']

# Initialize an empty dictionary to store MSE for each column
mse_scores = {}

# Loop over each column of interest to calculate MSE
for column in columns_of_interest:
    # Extract the true values from the test set and the predicted values
    true_values = aquifer_doganella_test_df[column][:10]  # Only the first 5 values to match the forecast length
    predicted_values = aquifer_doganella_predictions[column]
    
    # Calculate MSE and store it in the dictionary
    mse = mean_squared_error(true_values, predicted_values)
    mse_scores[column] = mse

# Print MSE scores
for column, mse in mse_scores.items():
    print(f"MSE for {column}: {mse}")

MSE for Depth_to_Groundwater_Pozzo_1: 1.0855508113644574
MSE for Depth_to_Groundwater_Pozzo_2: 0.5577649628075025
MSE for Depth_to_Groundwater_Pozzo_3: 9.60536439145728
MSE for Depth_to_Groundwater_Pozzo_4: 0.395458179791409
MSE for Depth_to_Groundwater_Pozzo_5: 0.001232688719572258
MSE for Depth_to_Groundwater_Pozzo_6: 0.9549690923042007
MSE for Depth_to_Groundwater_Pozzo_7: 0.07305525398805149
MSE for Depth_to_Groundwater_Pozzo_8: 0.24463434202960443
MSE for Depth_to_Groundwater_Pozzo_9: 0.6722268713218293


In [32]:
# AQUIFER LUCO


# 1. Train the VAR model on the entire DataFrame
aquifer_luco_model = VAR(aquifer_luco_df)
aquifer_luco_model_fitted = aquifer_luco_model.fit(maxlags=15, ic='aic')  # Example: using AIC to select optimal lag

# 2. Use the last 'lag_order' observations from the entire DataFrame for forecasting
aquifer_luco_lag_order = aquifer_luco_model_fitted.k_ar
aquifer_luco_last_obs = aquifer_luco_df.values[-aquifer_luco_lag_order:]

# Forecast the next 10 steps using the entire dataset
aquifer_luco_forecast = aquifer_luco_model_fitted.forecast(aquifer_luco_last_obs, steps=10)

# 3. Generate the forecast index
aquifer_luco_forecast_index = pd.date_range(start=aquifer_luco_train_df.index[-1], periods=10, freq='D')

# Convert the forecast to a DataFrame for easier interpretation
aquifer_luco_forecast_df = pd.DataFrame(aquifer_luco_forecast, index=aquifer_luco_forecast_index, columns=aquifer_luco_train_df.columns)

# Extract forecasts for the variables of interest
aquifer_luco_predictions = aquifer_luco_forecast_df['Depth_to_Groundwater_Podere_Casetta']
print(aquifer_luco_predictions)

2018-12-21   -7.680753
2018-12-22   -7.674557
2018-12-23   -7.558993
2018-12-24   -7.664035
2018-12-25   -7.667185
2018-12-26   -7.616886
2018-12-27   -7.615110
2018-12-28   -7.562495
2018-12-29   -7.587447
2018-12-30   -7.490169
Freq: D, Name: Depth_to_Groundwater_Podere_Casetta, dtype: float64


/home/codespace/.python/current/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:559: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  _index = to_datetime(index)
/home/codespace/.python/current/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [35]:
# EVALUATE AQUIFER LUCO MODEL

# Column of interest
column_of_interest = 'Depth_to_Groundwater_Podere_Casetta'

# Extract the true values from the test set for the column of interest
true_values = aquifer_luco_test_df[column_of_interest][:10]  # Assuming you want to match the forecast length

# Since aquifer_luco_predictions is a Series, it already contains the predicted values
predicted_values = aquifer_luco_predictions

# Calculate MSE for the column of interest
mse = mean_squared_error(true_values, predicted_values)

# Print the MSE score for the column
print(f"MSE for {column_of_interest}: {mse}")

MSE for Depth_to_Groundwater_Podere_Casetta: 0.0036516317221499265
